In [ ]:
!pip install transformers torch datasets transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/'

Mounted at /content/drive


In [ ]:
def json_to_df(file_name):
    with open(file_path+file_name, 'r',encoding='UTF-8') as f:
        data = json.load(f)

    df = pd.json_normalize(data, record_path = ['sentences'], meta=['id'], record_prefix='sent_')
    df = df.drop(columns = ['sent_id', 'sent_speaker', 'sent_types', 'sent_intensity_sum', 'id',
                            'sent_text', 'sent_intensity', 'sent_votes', 'sent_frame_id', 'sent_mapped_slots'])

    return df

In [ ]:
import json
import pandas as pd

merged_df = pd.DataFrame()

for i in range(1, 7):
    merged_df = pd.concat([merged_df, json_to_df(f'talksets-train-{i}.json')])

merged_df.reset_index(drop = True, inplace=True)
merged_df.head(10)

,sent_origin_text,sent_types
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,[CENSURE]
1,맞아. 사람들이 진짜 개념이없다,[CENSURE]
2,저렇게는 살지 말아야지,[CENSURE]
3,그러게 게으르고 멍청한 사람들은 맞아야해,[CENSURE]
4,인방 보는 남자는 거르는게 맞다,[CENSURE]
5,특히 벗방보는 애들은 진짜 거세 시켜야함,"[HATE, CENSURE, SEXUAL]"
6,볼 게 없어서 벗방을 보냐 ㅋㅋㅋ,"[CENSURE, SEXUAL]"
7,사회성이 얼마나 떨어지면 그러냐 ㅋㅋㅋ,[CENSURE]
8,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"[HATE, CENSURE]"
9,이래서 인방충~ 인방충~ 하는거 구나,"[HATE, CENSURE]"


In [ ]:
for idx, row in merged_df.iterrows():
    sent, labels = row
    temp = []

    if 'IMMORAL_NONE' in labels:
        temp.append(0)

    if 'CENSURE' in labels:
        temp.append(1)

    if 'DISCRIMINATION' in labels:
        temp.append(2)

    if 'HATE' in labels:
        temp.append(3)

    if 'VIOLENCE' in labels:
        temp.append(4)

    if 'CRIME' in labels:
        temp.append(5)

    if 'SEXUAL' in labels:
        temp.append(6)

    if 'ABUSE' in labels:
        temp.append(7)

    temp = sorted(temp)
    temp = list(map(str, temp))
    temp = ','.join(temp)

    merged_df.loc[idx, 'label'] = temp

merged_df.drop(columns = 'sent_types', inplace = True)
merged_df.head(10)

,sent_origin_text,label
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,1
1,맞아. 사람들이 진짜 개념이없다,1
2,저렇게는 살지 말아야지,1
3,그러게 게으르고 멍청한 사람들은 맞아야해,1
4,인방 보는 남자는 거르는게 맞다,1
5,특히 벗방보는 애들은 진짜 거세 시켜야함,"1,3,6"
6,볼 게 없어서 벗방을 보냐 ㅋㅋㅋ,"1,6"
7,사회성이 얼마나 떨어지면 그러냐 ㅋㅋㅋ,1
8,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"1,3"
9,이래서 인방충~ 인방충~ 하는거 구나,"1,3"


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408369 entries, 0 to 408368
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   sent_origin_text  408369 non-null  object
 1   label             408369 non-null  object
dtypes: object(2)
memory usage: 6.2+ MB


In [ ]:
merged_df.to_csv(file_path+'merge_data.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
from sklearn.model_selection import train_test_split

train_df = merged_df[merged_df.label.str.contains(',')]
merged_df = merged_df.drop(train_df.index)
train_df

,sent_origin_text,label
5,특히 벗방보는 애들은 진짜 거세 시켜야함,"1,3,6"
6,볼 게 없어서 벗방을 보냐 ㅋㅋㅋ,"1,6"
8,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"1,3"
9,이래서 인방충~ 인방충~ 하는거 구나,"1,3"
11,에로배우들을 데려다가 아이돌을 만드네,"1,6"
...,...,...
408347,공공기관부터 썩어빠졌으니 ㅋㅋ 진짜 쓰레기같은것들 한번 물갈이 해야해,"1,3"
408348,집에 처박혀서 애니나 보는 니 인생이 레전드,"1,2,3"
408349,밖에 나가봤자 친구 없는 니 인생 개불쌍,"1,2,3"
408358,그니까; 진짜 잡아서 날개 뜯고 머가리 뽑아버리고싶어.,"4,7"


In [ ]:
merged_df.label.value_counts()

0    183292
1    102485
3      7592
6      7239
4      4465
2      4435
7      3460
5      1316
Name: label, dtype: int64

In [ ]:
train_df.label.value_counts()

1,3            26718
1,2            12768
1,2,3          11341
1,7             8219
1,6             6129
               ...  
2,3,5,7            2
1,2,3,5,6,7        2
1,2,5,6,7          2
2,5,7              1
3,4,6,7            1
Name: label, Length: 110, dtype: int64

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314284 entries, 0 to 408367
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   sent_origin_text  314284 non-null  object
 1   label             314284 non-null  object
dtypes: object(2)
memory usage: 7.2+ MB


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(merged_df['sent_origin_text'], merged_df['label'], random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

In [ ]:
train_df = pd.concat([train_df, pd.DataFrame({'sent_origin_text': X_train, 'label': y_train})])
train_df

,sent_origin_text,label
5,특히 벗방보는 애들은 진짜 거세 시켜야함,"1,3,6"
6,볼 게 없어서 벗방을 보냐 ㅋㅋㅋ,"1,6"
8,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"1,3"
9,이래서 인방충~ 인방충~ 하는거 구나,"1,3"
11,에로배우들을 데려다가 아이돌을 만드네,"1,6"
...,...,...
205746,헛소리하지마 그거 하나로 니 인생 달라지지 않아,1
123650,요즘도 돈 뺏는 일진들 있어?,0
391397,취향이 너무 올드한거 아니야?,0
178070,"미개한 놈들, 저 넓은 도로에 신호등도 없네.",3


In [ ]:
test_df = pd.DataFrame({'sent_origin_text': X_test, 'label': y_test})
val_df = pd.DataFrame({'sent_origin_text': X_val, 'label': y_val})

print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(270869, 2)
(78571, 2)
(58929, 2)


In [ ]:
def one_hot_encode(x):
    x = str(x)

    label_vector = np.zeros((8, ))
    labels = list(map(int, x.split(',')))
    label_vector[labels] = 1

    return label_vector

print(one_hot_encode('1,3'))
print(one_hot_encode('1,5'))
print(one_hot_encode('1,6,7'))

[0. 1. 0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 1.]


In [ ]:
for idx, row in train_df.iterrows():
    one_hot_label = one_hot_encode(row['label'])
    train_df.loc[idx, labels] = one_hot_label

In [ ]:
for idx, row in test_df.iterrows():
    one_hot_label = one_hot_encode(row['label'])
    test_df.loc[idx, labels] = one_hot_label

In [ ]:
for idx, row in val_df.iterrows():
    one_hot_label = one_hot_encode(row['label'])
    val_df.loc[idx, labels] = one_hot_label

In [ ]:
train_df.head(10)

,sent_origin_text,label,IMMORAL_NONE,CENSURE,DISCRIMINATION,HATE,VIOLENCE,CRIME,SEXUAL,ABUSE
5,특히 벗방보는 애들은 진짜 거세 시켜야함,"1,3,6",0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
6,볼 게 없어서 벗방을 보냐 ㅋㅋㅋ,"1,6",0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"1,3",0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
9,이래서 인방충~ 인방충~ 하는거 구나,"1,3",0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
11,에로배우들을 데려다가 아이돌을 만드네,"1,6",0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
12,쟤네들은 표정이 딱 야동인데..,"1,6",0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13,솔직히 여자 48kg이상은 나가 디져야 하는 거 아님,"1,3",0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
16,중국은 두 발달린 짐승,"1,3",0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
18,짱깨는 반드시 해체되어야 함,"1,3",0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
26,줫빠지게 힘들어서 잣같았다,"6,7",0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
test_df.head(10)

,sent_origin_text,label,IMMORAL_NONE,CENSURE,DISCRIMINATION,HATE,VIOLENCE,CRIME,SEXUAL,ABUSE
397071,뭐래 뼈 위치 재배치해줘?,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
56007,결국 본성을 드러냈구만 저 자식,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
289340,돌려까는 게 아니라 둘이 친해서 장난치는 거야.,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219437,최근에 여자경찰이 취객 잡는 영상 봤어?,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121052,한남들 양남들에게 차마 뭐라 못하니까 양남 만나는 여자 까는 거 진짜 개찐따 같다.,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
92266,일찍가서 가운데 자리로 가봐 가운데는 잘봐주시더라,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182200,사달라고 한 적 없는데 뭐지,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243169,아는 형이 #@# 다니는데 연봉 5천 중반에 성과금이랑 주식 받는 거 이것저것 합치...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208414,머갈통 부숴놓기 전에 말 좀 들어,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
395524,아니야 그런 시선으로 그분들 바라보는건 안좋다고 생각해,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
val_df.head(10)

,sent_origin_text,label,IMMORAL_NONE,CENSURE,DISCRIMINATION,HATE,VIOLENCE,CRIME,SEXUAL,ABUSE
198206,애플워치 사고 운동이나 할까?,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82170,그거 안경 혐오 발언임.,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108734,그냥 정치인들이 잊을만하면 들고 나와서 표팔이 하는거지 ㅋㅋㅋ,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205327,꼴페미들 보들보들 거리는거 볼 수 있겠네 ㅋㅋㅋ,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
240438,한가지 더 있네. 부모한테 말대꾸하는 버르장머리도,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
233956,밥은 먹을수 있게 정도껏해야지..,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109697,어차피 작아서 버릴 건데. 되게 뭐라하네. 남일에 관심 껐으면.,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
190600,근데 이건 예쁜 분홍색 지갑이라 정말 마음에 들어.,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269069,존박아 니가 설래발치니 요모양 요꼴이잔아.붕신,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
292303,무슨 냄새가 난다그래~좋기만하구만,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from datasets import Dataset, DatasetDict

intent_ds = DatasetDict({'train': Dataset.from_pandas(train_df), 'val': Dataset.from_pandas(val_df), 'test': Dataset.from_pandas(test_df)})
intent_ds

DatasetDict({
    train: Dataset({
        features: ['sent_origin_text', 'label', 'IMMORAL_NONE', 'CENSURE', 'DISCRIMINATION', 'HATE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE', '__index_level_0__'],
        num_rows: 270869
    })
    val: Dataset({
        features: ['sent_origin_text', 'label', 'IMMORAL_NONE', 'CENSURE', 'DISCRIMINATION', 'HATE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE', '__index_level_0__'],
        num_rows: 78571
    })
    test: Dataset({
        features: ['sent_origin_text', 'label', 'IMMORAL_NONE', 'CENSURE', 'DISCRIMINATION', 'HATE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE', '__index_level_0__'],
        num_rows: 58929
    })
})

In [ ]:
train_df.to_csv(file_path+'train_data.csv', encoding = 'utf-8-sig', index = False)
val_df.to_csv(file_path+'validation_data.csv', encoding = 'utf-8-sig', index = False)
test_df.to_csv(file_path+'test_data.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
from transformers import AutoTokenizer
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU or CPU
print(f'Using {device} device...')

model_ckpt = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Using cuda device...


In [ ]:
id2label = {0: 'IMMORAL_NONE',
            1: 'CENSURE',
            2: 'DISCRIMINATION',
            3: 'HATE',
            4: 'VIOLENCE',
            5: 'CRIME',
            6: 'SEXUAL',
            7: 'ABUSE'
            }
label2id = {v: k for k, v in id2label.items()}
labels = list(label2id.keys())

In [ ]:
# 토큰 분리를 위한 함수 정의
def tokenize(batch):
    model_inputs = tokenizer(batch['sent_origin_text'])

    # add labels
    labels_batch = {k: batch[k] for k in batch.keys() if k in labels}

    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(batch), len(labels)))

    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    model_inputs['labels'] = labels_matrix.tolist()

    return model_inputs

In [ ]:
import numpy as np

encoded = intent_ds.map(tokenize, batched=True, batch_size=None, remove_columns=intent_ds['train'].column_names)
encoded

Map:   0%|          | 0/270869 [00:00<?, ? examples/s]

ValueError: ignored

In [ ]:
intent_ds

DatasetDict({
    train: Dataset({
        features: ['sent_origin_text', 'label', 'IMMORAL_NONE', 'CENSURE', 'DISCRIMINATION', 'HATE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE', '__index_level_0__'],
        num_rows: 270869
    })
    val: Dataset({
        features: ['sent_origin_text', 'label', 'IMMORAL_NONE', 'CENSURE', 'DISCRIMINATION', 'HATE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE', '__index_level_0__'],
        num_rows: 78571
    })
    test: Dataset({
        features: ['sent_origin_text', 'label', 'IMMORAL_NONE', 'CENSURE', 'DISCRIMINATION', 'HATE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE', '__index_level_0__'],
        num_rows: 58929
    })
})

In [ ]:
labels_batch = {k: intent_ds['train'][k] for k in intent_ds.keys() if k in labels}
labels_batch

{}

In [ ]:
intent_ds.keys()

dict_keys(['train', 'val', 'test'])

In [ ]:
ta = np.array(encoded['train']['labels'])
ta.shape

In [ ]:
tb = np.array(encoded['test']['labels'])
tb.shape

In [ ]:
tc = np.array(encoded['val']['labels'])
tc.shape

In [ ]:
td = np.array(encoded['train']['input_ids'])
td.shape

In [ ]:
te = np.array(encoded['test']['input_ids'])
te.shape

In [ ]:
tf = np.array(encoded['val']['input_ids'])
tf.shape

In [ ]:
print(encoded['train']['labels'][0])

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id).to(device)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

batch_size = 128
logging_steps = len(encoded["train"]) // batch_size
model_name = file_path + f"{model_ckpt}-finetuned-intent"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  log_level="error")

In [ ]:
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=encoded["train"],
                  eval_dataset=encoded["val"],
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
                  tokenizer=tokenizer)
trainer.train();